# Task 1: Get the Info and Load the Web Page

### Import the Necessary Libraries

In [495]:
from bs4 import BeautifulSoup as bs
import re
import requests
import pandas as pd

### Load our first page

In [496]:
#Load the webpage
r = requests.get("https://pt.wikipedia.org/wiki/Big_Brother_Brasil_1")

#BeautifulSoup convertion
soup = bs(r.content)

#Print our HTML
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Big Brother Brasil 1 – Wikipédia, a enciclopédia livre
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-fe

### Setting the Information that i want to get

In [497]:
urls = [
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_1",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_2",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_3",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_4",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_5",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_6",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_7",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_8",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_9",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_10",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_11",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_12",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_13",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_14",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_15",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_16",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_17",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_18",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_19",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_20",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_21",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_22",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_23",
    "https://pt.wikipedia.org/wiki/Big_Brother_Brasil_24"
]

info_table = soup.find(class_="wikitable")
info_rows = info_table.find_all("tr")
all_info_tables = soup.find_all(class_="wikitable") #To store all the wikitables, this gonna be useful late :)

# Task 2: Creating Functions to Extract the Data

### A Function to Extract the 'wikitable' from All Types of Sources

### Getting the Name and Nicknames

In [498]:
def extract_participants_names_and_nicknames(info_rows):
    participants_name = []
    nicknames = []
    for row in info_rows[1:]:
        name_text = row.get_text().strip().split('\n')[0]
        # Remover 'Big Brother Brasil' do nome, se presente
        cleaned_name = name_text.replace('Big Brother Brasil', '').strip()
        # Encontrar e extrair apelido se presente
        match = re.search(r'\(([^()]+)\)', cleaned_name)
        if match:
            nickname = match.group(1).strip()
            nicknames.append(nickname)
            # Remover o apelido do nome completo
            cleaned_name = re.sub(r'\s*\([^()]+\)', '', cleaned_name).strip()
        else:
            nicknames.append(None)
        participants_name.append(cleaned_name)
    return participants_name, nicknames

In [499]:
extract_participants_names(info_rows)

['Kleber de Paula Pedra',
 'Vanessa Melanie Pascale Ekypenyong Walborn',
 'André Batista de Carvalho',
 'Antonio Sérgio Tavares Campo',
 'Alessandra Begliomini',
 'Estela Padilha',
 'Adriano Luiz Ramos de Castro',
 'Helena Louro',
 'Cristiana Carvalho Mota Machado',
 'Bruno Saladini Trautvetter',
 'Cristiane Souza Dantas',
 'Caetano Zonaro']

### Getting the Date of Birth

In [500]:
def date_birth(info_rows):
    participants_date = []
    for row in info_rows[1:]:
        try:
            date_text = row.get_text().split('\n\n')[1]
            date_text = date_text.split('[nota')[0].strip() #to remove the 'note' on the 'data_lista'
            if date_text == "?": #to remove the '?' on the 'data_lista'
                    participants_date.append("Data de nascimento não disponível")
            else:
                participants_date.append(date_text)
        except IndexError:
            participants_date.append("Data de nascimento não disponível")
    return participants_date

In [501]:
date_birth(info_rows)

['14/02/1978',
 '28/11/1974',
 '28/11/1974',
 '06/12/1972',
 '05/12/1974',
 '27/09/1978',
 '23/10/1968',
 '21/06/1976',
 '07/12/1974',
 '10/03/1976',
 '21/12/1975',
 '20/06/1961']

### Getting the Ocupation

In [502]:
def ocupation(info_rows):
    participants_ocupation = []
    for row in info_rows[1:]:
        try:
            occupation_text = row.get_text().split('\n\n')[2]
            participants_ocupation.append(occupation_text)
        except IndexError:
            participants_ocupation.append("Ocupação não disponível")
    return participants_ocupation

In [503]:
ocupation(info_rows)

['Dançarino',
 'Modelo',
 'Professor de canto',
 'Cabeleireiro',
 'Empresária',
 'Videografista',
 'Artista plástico',
 'Atriz',
 'Cantora',
 'Modelo',
 'Barwoman',
 'Modelo']

### Getting the Origin

In [504]:
def origin_of(info_rows):
    participants_origin = []
    for row in info_rows[1:]:
        try:
            origin_text = row.get_text().split('\n\n')[3]
            participants_origin.append(origin_text)
        except IndexError:
            participants_origin.append("Origem não disponível")
    return participants_origin

In [505]:
origin_of(info_rows)

['Campinas, São Paulo',
 'Rio de Janeiro, Rio de Janeiro',
 'Rio de Janeiro, Rio de Janeiro',
 'Luanda, Luanda, Angola ',
 'São Paulo, São Paulo',
 'São Paulo, São Paulo',
 'Salvador, Bahia',
 'Maringá, Paraná',
 'Rio de Janeiro, Rio de Janeiro',
 'Rio de Janeiro, Rio de Janeiro',
 'Rio de Janeiro, Rio de Janeiro',
 'São Paulo, São Paulo']

### Getting the Result

In [506]:
def result_of(info_rows):
    participants_result = []
    for row in info_rows[1:]:
        try:
            result_text = row.get_text().split('\n\n')[4].replace('em', ' em').replace('Vencedor', '1 lugar')
            participants_result.append(result_text)
        except IndexError:
            participants_result.append("Resultado não disponível")
    return participants_result

In [507]:
result_of(info_rows)

['1 lugar em 2 de abril de 2002',
 '2º lugar em 2 de abril de 2002',
 '3º lugar em 2 de abril de 2002',
 '9º eliminado em 31 de março de 2002',
 '8ª eliminada em 26 de março de 2002',
 '7ª eliminada em 19 de março de 2002',
 '6º eliminado em 12 de março de 2002',
 '5ª eliminada em 5 de março de 2002',
 '4ª eliminada em 26 de fevereiro de 2002',
 '3º eliminado em 19 de fevereiro de 2002',
 '2ª eliminada em 12 de fevereiro de 2002',
 '1º eliminado em 5 de fevereiro de 2002']

### Getting the Sign

In [508]:
def calcular_signos(info_rows):
    def date_birth(info_rows):
        participants_date = []
        for row in info_rows[1:]:
            try:
                date_text = row.get_text().split('\n\n')[1]
                participants_date.append(date_text)
            except IndexError:
                participants_date.append("Data de nascimento não disponível")
        return participants_date
    
    participants_sign = []
    for data_nascimento in date_birth(info_rows):
        try:
            dia, mes, _ = data_nascimento.split("/")
            
            if (mes == "03" and int(dia) >= 21) or (mes == "04" and int(dia) <= 19):
                signo_atual = "Áries"
            elif (mes == "04" and int(dia) >= 20) or (mes == "05" and int(dia) <= 20):
                signo_atual = "Touro"
            elif (mes == "05" and int(dia) >= 21) or (mes == "06" and int(dia) <= 20):
                signo_atual = "Gêmeos"
            elif (mes == "06" and int(dia) >= 21) or (mes == "07" and int(dia) <= 22):
                signo_atual = "Câncer"
            elif (mes == "07" and int(dia) >= 23) or (mes == "08" and int(dia) <= 22):
                signo_atual = "Leão"
            elif (mes == "08" and int(dia) >= 23) or (mes == "09" and int(dia) <= 22):
                signo_atual = "Virgem"
            elif (mes == "09" and int(dia) >= 23) or (mes == "10" and int(dia) <= 22):
                signo_atual = "Libra"
            elif (mes == "10" and int(dia) >= 23) or (mes == "11" and int(dia) <= 21):
                signo_atual = "Escorpião"
            elif (mes == "11" and int(dia) >= 22) or (mes == "12" and int(dia) <= 21):
                signo_atual = "Sagitário"
            elif (mes == "12" and int(dia) >= 22) or (mes == "01" and int(dia) <= 19):
                signo_atual = "Capricórnio"
            elif (mes == "01" and int(dia) >= 20) or (mes == "02" and int(dia) <= 18):
                signo_atual = "Aquário"
            elif (mes == "02" and int(dia) >= 19) or (mes == "03" and int(dia) <= 20):
                signo_atual = "Peixes"
            else:
                signo_atual = "Data de nascimento inválida"

            participants_sign.append(signo_atual)
        except ValueError:
            participants_sign.append("Data de nascimento inválida")
    
    return participants_sign

In [509]:
calcular_signos(info_rows)

['Aquário',
 'Sagitário',
 'Sagitário',
 'Sagitário',
 'Sagitário',
 'Libra',
 'Escorpião',
 'Câncer',
 'Sagitário',
 'Peixes',
 'Sagitário',
 'Gêmeos']

### Getting the Number of Votes

In [510]:
def get_votes_from_wikitable(tables):
    votes = []
    
    if tables:
        # Extrair as células da tabela com o atributo 'bgcolor'
        table_cells = tables.find_all("td", {"bgcolor": True})
        
        for cell in table_cells:
            text = cell.get_text()
            # Extrair números de 1 a 99
            numbers = [int(char) for char in text.split() if char.isdigit() and 1 <= int(char) <= 99]
            votes.extend(numbers)
    
    return votes

# Task 3: Apply the Functions on the URLs

### Function to Extract the 'wikitable' from all URLs | table_index is where the table is located

In [511]:
def extrair_tabela_wikipedia(urls, table_index=0):
    all_info_rows = {}
    
    for url in urls:
        # Carregar a página da web
        r = requests.get(url)

        # Converter para BeautifulSoup
        soup = bs(r.content, 'html.parser')

        # Encontrar todas as tabelas
        info_tables = soup.find_all(class_="wikitable")
        
        # Verificar se a tabela foi encontrada
        if info_tables:
            # Verificar se há tabelas suficientes
            if len(info_tables) > table_index:
                # Extrair as linhas da tabela desejada
                info_rows = info_tables[table_index].find_all("tr")
                
                # Remover as linhas que contêm a palavra "Repescagem"
                filtered_rows = [row for row in info_rows if 'Repescagem' not in row.get_text()]
                all_info_rows[url] = filtered_rows
            else:
                print(f"A tabela de índice {table_index} não foi encontrada para {url}.")
                all_info_rows[url] = None
        else:
            print(f"Tabela não encontrada para {url}.")
            all_info_rows[url] = None

    return all_info_rows

### Getting the Number of the Season | Setting the table_index to get the right wikitable

In [512]:
# Função para extrair o número da temporada a partir da URL
def extract_season_number(url):
    match = re.search(r'Big_Brother_Brasil_(\d+)', url)
    if match:
        return int(match.group(1))
    return None

In [513]:
todas_as_info_rows = {}

for url in urls:
    season_number = extract_season_number(url)
    
    if season_number in {3, 4, 10, 11}:
        tabela = extrair_tabela_wikipedia([url], table_index=1)
    elif season_number in {7, 9, 12}:
        tabela = extrair_tabela_wikipedia([url], table_index=2)
    elif season_number in {13, 14}:
        tabela = extrair_tabela_wikipedia([url], table_index=3)
    elif season_number in {15, 17, 19, 20}:
        tabela = extrair_tabela_wikipedia([url], table_index=4)
    elif season_number in {16, 21, 22}:
        tabela = extrair_tabela_wikipedia([url], table_index=5)
    elif season_number == 18:
        tabela = extrair_tabela_wikipedia([url], table_index=6)
    elif season_number == 23:
        tabela = extrair_tabela_wikipedia([url], table_index=10)
    elif season_number == 24:
        tabela = extrair_tabela_wikipedia([url], table_index=11) 
        
    else:
        tabela = extrair_tabela_wikipedia([url])
    
    # Atualizar o dicionário todas_as_info_rows com os dados extraídos
    todas_as_info_rows.update(tabela)


### Now Let's Get All the Data

In [524]:
# Listas para armazenar os dados
nomes_lista = []
datas_lista = []
ocupacoes_lista = []
origens_lista = []
resultados_lista = []
signos_lista = []
apelidos_lista = []

# Aplicar os filtros para cada URL
for url, info_rows in todas_as_info_rows.items():
    season_number = extract_season_number(url)
    if info_rows:
        # Extraindo nomes e apelidos dos participantes
        nomes, apelidos = extract_participants_names_and_nicknames(info_rows)
        
        nomes_lista.extend([(season_number, nome) for nome in nomes])
        
        apelidos_lista.extend([(season_number, apelido) for apelido in apelidos])

        # Extraindo datas de nascimento
        datas = date_birth(info_rows)
        datas_lista.extend([(season_number, data) for data in datas])

        # Extraindo ocupações
        ocupacoes = ocupation(info_rows)
        ocupacoes_lista.extend([(season_number, ocupacao) for ocupacao in ocupacoes])

        # Extraindo origens
        origens = origin_of(info_rows)
        origens_lista.extend([(season_number, origem) for origem in origens])

        # Extraindo resultados
        resultados = result_of(info_rows)
        resultados_lista.extend([(season_number, resultado) for resultado in resultados])

        # Calculando signos
        signos = calcular_signos(info_rows)
        signos_lista.extend([(season_number, signo) for signo in signos])
    else:
        print(f"Sem dados para {url}")

In [515]:
nomes_lista #All the Names

[(1, 'Kleber de Paula Pedra'),
 (1, 'Vanessa Melanie Pascale Ekypenyong Walborn'),
 (1, 'André Batista de Carvalho'),
 (1, 'Antonio Sérgio Tavares Campo'),
 (1, 'Alessandra Begliomini'),
 (1, 'Estela Padilha'),
 (1, 'Adriano Luiz Ramos de Castro'),
 (1, 'Helena Louro'),
 (1, 'Cristiana Carvalho Mota Machado'),
 (1, 'Bruno Saladini Trautvetter'),
 (1, 'Cristiane Souza Dantas'),
 (1, 'Caetano Zonaro'),
 (2, 'Rodrigo Fraga Leonel'),
 (2, 'Manuela Santos Saadeh'),
 (2, 'Maria Aparecida Junqueira de Moraes'),
 (2, 'Thyrso Mattos Martins'),
 (2, 'Tarciana de Lima Mafra'),
 (2, 'Fabrício\xa0Luiz do Amaral'),
 (2, 'Thaís Ventura Pugliese'),
 (2, 'Moisés Joaquim da Silva'),
 (2, 'Jeferson de Oliveira dos Santos'),
 (2, 'Fernando Fernandes de Pádua'),
 (2, 'Vanessa Cristina Soares Dias'),
 (2, 'Rita Sinara Gonçalves da Silva Lopes'),
 (3, 'André Augusto Ferreira Fontes'),
 (3, 'Elane Silva Chaves'),
 (3, 'Viviane da Silva Oliveira'),
 (3, 'Jean Massumi Hara'),
 (3, 'Harry Grossman'),
 (3, 'Sabri

In [516]:
datas_lista #All the date of birth

[(1, '14/02/1978'),
 (1, '28/11/1974'),
 (1, '28/11/1974'),
 (1, '06/12/1972'),
 (1, '05/12/1974'),
 (1, '27/09/1978'),
 (1, '23/10/1968'),
 (1, '21/06/1976'),
 (1, '07/12/1974'),
 (1, '10/03/1976'),
 (1, '21/12/1975'),
 (1, '20/06/1961'),
 (2, '04/04/1970'),
 (2, '31/05/1979'),
 (2, '30/10/1963'),
 (2, '04/10/1975'),
 (2, '08/11/1974'),
 (2, '16/05/1977'),
 (2, '08/03/1983'),
 (2, '25/09/1975'),
 (2, '21/10/1974'),
 (2, '28/03/1981'),
 (2, '11/06/1980'),
 (2, '13/08/1972'),
 (3, '27/09/1972'),
 (3, '24/08/1984'),
 (3, '11/02/1975'),
 (3, '16/09/1974'),
 (3, '15/01/1968'),
 (3, '04/02/1981'),
 (3, '24/06/1972'),
 (3, '21/10/1976'),
 (3, '03/05/1982'),
 (3, '04/09/1964'),
 (3, '05/04/1980'),
 (3, '10/04/1981'),
 (3, '18/05/1975'),
 (3, '14/10/1969'),
 (3, '26/05/1975'),
 (4, '26/02/1983'),
 (4, '08/06/1982'),
 (4, '09/07/1980'),
 (4, '18/06/1978'),
 (4, '22/03/1978'),
 (4, '12/08/1978'),
 (4, '29/04/1972'),
 (4, '27/12/1980'),
 (4, '11/12/1971'),
 (4, '14/11/1973'),
 (4, '22/10/1982'),


In [517]:
origens_lista #All the Origins

[(1, 'Campinas, São Paulo'),
 (1, 'Rio de Janeiro, Rio de Janeiro'),
 (1, 'Rio de Janeiro, Rio de Janeiro'),
 (1, 'Luanda, Luanda, Angola '),
 (1, 'São Paulo, São Paulo'),
 (1, 'São Paulo, São Paulo'),
 (1, 'Salvador, Bahia'),
 (1, 'Maringá, Paraná'),
 (1, 'Rio de Janeiro, Rio de Janeiro'),
 (1, 'Rio de Janeiro, Rio de Janeiro'),
 (1, 'Rio de Janeiro, Rio de Janeiro'),
 (1, 'São Paulo, São Paulo'),
 (2, 'Ribeirão Preto, São Paulo'),
 (2, 'Rio de Janeiro, Rio de Janeiro'),
 (2, 'Rio de Janeiro, Rio de Janeiro'),
 (2, 'São Paulo, São Paulo'),
 (2, 'Recife, Pernambuco'),
 (2, 'Novo Hamburgo, Rio Grande do Sul'),
 (2, 'Niterói, Rio de Janeiro'),
 (2, 'Ipojuca, Pernambuco'),
 (2, 'Carapicuíba, São Paulo'),
 (2, 'São Paulo, São Paulo'),
 (2, 'São Paulo, São Paulo'),
 (2, 'Esteio, Rio Grande do Sul'),
 (3, 'Goiânia, Goiás'),
 (3, 'Itanhém, Bahia'),
 (3, 'Votorantim, São Paulo'),
 (3, 'São Paulo, São Paulo'),
 (3, 'Teresópolis, Rio de Janeiro'),
 (3, 'Penápolis, São Paulo'),
 (3, 'Londrina, Pa

In [518]:
resultados_lista

[(1, '1 lugar em 2 de abril de 2002'),
 (1, '2º lugar em 2 de abril de 2002'),
 (1, '3º lugar em 2 de abril de 2002'),
 (1, '9º eliminado em 31 de março de 2002'),
 (1, '8ª eliminada em 26 de março de 2002'),
 (1, '7ª eliminada em 19 de março de 2002'),
 (1, '6º eliminado em 12 de março de 2002'),
 (1, '5ª eliminada em 5 de março de 2002'),
 (1, '4ª eliminada em 26 de fevereiro de 2002'),
 (1, '3º eliminado em 19 de fevereiro de 2002'),
 (1, '2ª eliminada em 12 de fevereiro de 2002'),
 (1, '1º eliminado em 5 de fevereiro de 2002'),
 (2, '1 lugar em 23 de julho de 2002'),
 (2, '2º lugar em 23 de julho de 2002'),
 (2, '10ª eliminada em 21 de julho de 2002'),
 (2, '9º eliminado em 16 de julho de 2002'),
 (2, '8ª eliminada em 9 de julho de 2002'),
 (2, '7º eliminado em 2 de julho de 2002'),
 (2, '6ª eliminada em 25 de junho de 2002'),
 (2, '5º eliminado em 18 de junho de 2002'),
 (2, '4º eliminado em 11 de junho de 2002'),
 (2, '3º eliminado em 4 de junho de 2002'),
 (2, '2ª eliminada em 2

In [519]:
signos_lista

[(1, 'Aquário'),
 (1, 'Sagitário'),
 (1, 'Sagitário'),
 (1, 'Sagitário'),
 (1, 'Sagitário'),
 (1, 'Libra'),
 (1, 'Escorpião'),
 (1, 'Câncer'),
 (1, 'Sagitário'),
 (1, 'Peixes'),
 (1, 'Sagitário'),
 (1, 'Gêmeos'),
 (2, 'Áries'),
 (2, 'Gêmeos'),
 (2, 'Escorpião'),
 (2, 'Libra'),
 (2, 'Escorpião'),
 (2, 'Touro'),
 (2, 'Peixes'),
 (2, 'Libra'),
 (2, 'Libra'),
 (2, 'Áries'),
 (2, 'Gêmeos'),
 (2, 'Leão'),
 (3, 'Libra'),
 (3, 'Virgem'),
 (3, 'Aquário'),
 (3, 'Virgem'),
 (3, 'Capricórnio'),
 (3, 'Aquário'),
 (3, 'Câncer'),
 (3, 'Libra'),
 (3, 'Touro'),
 (3, 'Virgem'),
 (3, 'Áries'),
 (3, 'Áries'),
 (3, 'Touro'),
 (3, 'Libra'),
 (3, 'Gêmeos'),
 (4, 'Peixes'),
 (4, 'Gêmeos'),
 (4, 'Câncer'),
 (4, 'Gêmeos'),
 (4, 'Áries'),
 (4, 'Leão'),
 (4, 'Touro'),
 (4, 'Capricórnio'),
 (4, 'Sagitário'),
 (4, 'Escorpião'),
 (4, 'Libra'),
 (4, 'Touro'),
 (4, 'Touro'),
 (4, 'Áries'),
 (5, 'Peixes'),
 (5, 'Câncer'),
 (5, 'Virgem'),
 (5, 'Gêmeos'),
 (5, 'Áries'),
 (5, 'Câncer'),
 (5, 'Libra'),
 (5, 'Áries'),
 (5, 

In [525]:
apelidos_lista

[(1, 'Kleber Bambam'),
 (1, None),
 (1, 'André Gabeh'),
 (1, None),
 (1, None),
 (1, None),
 (1, None),
 (1, None),
 (1, None),
 (1, None),
 (1, 'Xaiane'),
 (1, None),
 (2, None),
 (2, None),
 (2, 'Cida'),
 (2, None),
 (2, None),
 (2, None),
 (2, None),
 (2, None),
 (2, None),
 (2, None),
 (2, 'Tina'),
 (2, None),
 (3, 'Dhomini'),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (3, None),
 (4, 'Cida'),
 (4, None),
 (4, None),
 (4, None),
 (4, None),
 (4, None),
 (4, None),
 (4, 'Zulu'),
 (4, None),
 (4, 'Géris'),
 (4, None),
 (4, None),
 (4, None),
 (4, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, 'Tatiane Pink'),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (5, None),
 (6, 'Mara'),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),
 (6, None),


In [521]:
import pandas as pd

# Verificação se todas as listas têm o mesmo tamanho
assert len(nomes_lista) == len(datas_lista) == len(ocupacoes_lista) == len(origens_lista) == len(resultados_lista) == len(signos_lista), "Listas de dados têm tamanhos diferentes!"

# Extração dos dados individuais
temporadas = [season for season, _ in nomes_lista]
nomes = [nome for _, nome in nomes_lista]
datas = [data for _, data in datas_lista]
ocupacoes = [ocupacao for _, ocupacao in ocupacoes_lista]
origens = [origem for _, origem in origens_lista]
resultados = [resultado for _, resultado in resultados_lista]
signos = [signo for _, signo in signos_lista]

# Construção do DataFrame
df = pd.DataFrame({
    'Temporada': temporadas,
    'Nome': nomes,
    'Data de Nascimento': datas,
    'Ocupação': ocupacoes,
    'Origem': origens,
    'Resultado': resultados,
    'Signo': signos
})

# Exibindo o DataFrame

In [522]:
df

,Temporada,Nome,Data de Nascimento,Ocupação,Origem,Resultado,Signo
0,1,Kleber de Paula Pedra,14/02/1978,Dançarino,"Campinas, São Paulo",1 lugar em 2 de abril de 2002,Aquário
1,1,Vanessa Melanie Pascale Ekypenyong Walborn,28/11/1974,Modelo,"Rio de Janeiro, Rio de Janeiro",2º lugar em 2 de abril de 2002,Sagitário
2,1,André Batista de Carvalho,28/11/1974,Professor de canto,"Rio de Janeiro, Rio de Janeiro",3º lugar em 2 de abril de 2002,Sagitário
3,1,Antonio Sérgio Tavares Campo,06/12/1972,Cabeleireiro,"Luanda, Luanda, Angola",9º eliminado em 31 de março de 2002,Sagitário
4,1,Alessandra Begliomini,05/12/1974,Empresária,"São Paulo, São Paulo",8ª eliminada em 26 de março de 2002,Sagitário
...,...,...,...,...,...,...,...
407,24,Nizam Hayek Abou Jokh,07/10/1991,Executivo de contas internacional,"São Paulo, São Paulo",4º eliminado em 21 de janeiro de 2024,Libra
408,24,Vanessa Lopes Ramalho,05/07/2001,Criadora de conteúdo,"Brasília, Distrito Federal",Desistente em 19 de janeiro de 2024,Câncer
409,24,Lucas Pizane Fernandes Sapucaia,21/12/2001,Músico,"Itaparica, Bahia",3º eliminado em 16 de janeiro de 2024,Sagitário
410,24,Thalyta Alves Nascimento,24/08/1997,Advogada,"Belo Horizonte, Minas Gerais",2ª eliminada em 14 de janeiro de 2024,Virgem


In [523]:
# Exportando o DataFrame para um arquivo CSV
output_file_path = 'participantes_temporadas.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8', sep=',')